# Multiple Logistic regression model of Beijing PM2.5

Machine Learning <br>
Chris Xu <br>
Data Source: https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data

import:

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import numpy.polynomial.polynomial as poly
%matplotlib inline

Data Analysis and Preprocessing

In [2]:
#names = ['NO.', 'year', 'month', 'day', 'hour', 'pm2.5', 'DEWP',
# 'TEMP', 'PRES', 'cbwd', 'Iws', 'Is', 'Ir']
df = pd.read_csv(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/00381/PRSA_data_2010.1.1-2014.12.31.csv')
# out of 43824 samples, there are 2067 missing pm2.5 values
df = df.dropna()
print(df.head(6))
# shuffle the data
df = df.sample(frac=1)
print(df.head(6))

    No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES cbwd   Iws  Is  Ir
24  25  2010      1    2     0  129.0   -16  -4.0  1020.0   SE  1.79   0   0
25  26  2010      1    2     1  148.0   -15  -4.0  1020.0   SE  2.68   0   0
26  27  2010      1    2     2  159.0   -11  -5.0  1021.0   SE  3.57   0   0
27  28  2010      1    2     3  181.0    -7  -5.0  1022.0   SE  5.36   1   0
28  29  2010      1    2     4  138.0    -7  -5.0  1022.0   SE  6.25   2   0
29  30  2010      1    2     5  109.0    -7  -6.0  1022.0   SE  7.14   3   0
          No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES cbwd    Iws  \
39877  39878  2014      7   20    13   67.0    24  34.0  1004.0   SE  25.93   
11222  11223  2011      4   13    14  131.0     6  24.0  1010.0   SE  12.07   
15504  15505  2011     10    9     0  351.0    14  15.0  1017.0   SE   1.78   
21191  21192  2012      6    1    23  151.0    17  18.0  1012.0   SE   4.47   
25659  25660  2012     12    5     3   13.0   -16  -3.0  1022.0   

In [3]:
#only take one sample a day in order to get rid of repeated data
df1 = df[df.hour == 12]
print(df1.head(6))
y = np.array(df1['cbwd'])
x = np.array(df1[['TEMP','PRES', 'Iws']])

print(y.shape)
n = y.shape[0]
#y = y.reshape(n,1)
print(y.shape)

print(x)
df.isnull().sum()
wind_direction = []
wind_direction_appear = {
    "cv" : 0,
    "NE" : 0,
    "NW" : 0,
    "SE" : 0,
}
for i in range(y.shape[0]):
    if y[i] not in wind_direction:
        wind_direction.append(y[i])
    wind_direction_appear[y[i]] += 1
print("there are {} kinds of wind derections: {}".format(len(wind_direction), wind_direction))
print("appearance of each directions are {}".format(wind_direction_appear))

          No  year  month  day  hour  pm2.5  DEWP  TEMP    PRES cbwd     Iws  \
34236  34237  2013     11   27    12   13.0   -29   0.0  1027.0   NW  231.10   
37236  37237  2014      4    1    12  121.0     3  21.0  1013.0   cv    3.57   
1884    1885  2010      3   20    12  156.0   -10   8.0  1010.0   NW   84.95   
2700    2701  2010      4   23    12   19.0   -10  19.0  1021.0   NW    7.15   
36660  36661  2014      3    8    12  205.0    -6   6.0  1023.0   cv    4.47   
3204    3205  2010      5   14    12  147.0     9  21.0  1015.0   SE   22.36   

       Is  Ir  
34236   0   0  
37236   0   0  
1884    0   0  
2700    0   0  
36660   0   0  
3204    0   0  
(1731,)
(1731,)
[[   0.   1027.    231.1 ]
 [  21.   1013.      3.57]
 [   8.   1010.     84.95]
 ...
 [   7.   1019.      5.37]
 [  26.   1004.      9.84]
 [  29.   1011.      7.6 ]]
there are 4 kinds of wind derections: ['NW', 'cv', 'SE', 'NE']
appearance of each directions are {'cv': 461, 'NE': 191, 'NW': 538, 'SE': 541}


Next, I want to predict the wind directions, north or south, with this dataset
using logistic regression <br>

Replace "cv", "SE" to 0 and "NE", "NW" to 1

In [4]:
for i in range(y.shape[0]):
    if y[i] == 'cv' or y[i] == 'SE':
        y[i] = 0
    elif y[i] == 'NE' or y[i] == 'NW':
        y[i] = 1
print(y[0:10])

[1 0 1 1 0 0 1 0 0 0]


logistic regression

In [5]:
x_scale = preprocessing.scale(x)
x_train, x_test, y_train, y_test = train_test_split(x_scale, y, test_size = 0.25)
y_train = y_train.reshape((y_train.shape[0]), 1)
y_train=y_train.astype('int')
y_test=y_test.astype('int')
print(x_train.shape)
print(y_train.shape)
print(x_train)

(1298, 3)
(1298, 1)
[[-1.78742436  2.17653794  0.47685174]
 [ 1.10514804 -1.63255862 -0.42419518]
 [-0.85159211  1.00450823 -0.40588623]
 ...
 [-1.95757567  2.66488365 -0.37811424]
 [ 0.67976975 -0.94887462  0.05410027]
 [ 0.08424014 -1.33955119 -0.27690075]]


In [6]:
regr = LogisticRegression(penalty='none', solver='sag')
regr.fit(x_train, y_train)

print('The intercept w0 = ', regr.intercept_)
print('The coefficients w[1..d]=', regr.coef_)
y_train_pred = regr.predict(x_train)
y_test_pred = regr.predict(x_test)
print(y_test_pred)

The intercept w0 =  [-0.09344046]
The coefficients w[1..d]= [[-0.29318875  0.1851941   1.778448  ]]
[1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0
 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]


c:\users\chrix\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


result without regularization

In [7]:
print(confusion_matrix(y_test, y_test_pred))
print(regr.score(x_test,y_test))
print(classification_report(y_test, y_test_pred))

[[232  18]
 [ 97  86]]
0.7344110854503464
              precision    recall  f1-score   support

           0       0.71      0.93      0.80       250
           1       0.83      0.47      0.60       183

    accuracy                           0.73       433
   macro avg       0.77      0.70      0.70       433
weighted avg       0.76      0.73      0.72       433



now try Ridge regularization

In [8]:
regr = LogisticRegression(penalty='l2', solver='sag')
regr.fit(x_train, y_train)

print('The intercept w0 = ', regr.intercept_)
print('The coefficients w[1..d]=', regr.coef_)
y_train_pred = regr.predict(x_train)
y_test_pred = regr.predict(x_test)
print(y_test_pred)

The intercept w0 =  [-0.10543109]
The coefficients w[1..d]= [[-0.29433847  0.1832596   1.72249577]]
[1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0
 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]


c:\users\chrix\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


result with l2 regularization

In [9]:
print(confusion_matrix(y_test, y_test_pred))
print(regr.score(x_test,y_test))
print(classification_report(y_test, y_test_pred))


[[231  19]
 [ 98  85]]
0.7297921478060047
              precision    recall  f1-score   support

           0       0.70      0.92      0.80       250
           1       0.82      0.46      0.59       183

    accuracy                           0.73       433
   macro avg       0.76      0.69      0.70       433
weighted avg       0.75      0.73      0.71       433



now try Lasso regularization <br>
used lbfgs solver for the previous model, switch to saga next because
lbfgs does not support l1

In [10]:
regr = LogisticRegression(penalty='l1', solver='saga')
regr.fit(x_train, y_train)

print('The intercept w0 = ', regr.intercept_)
print('The coefficients w[1..d]=', regr.coef_)
y_train_pred = regr.predict(x_train)
y_test_pred = regr.predict(x_test)
print(y_test_pred)

The intercept w0 =  [-0.10032983]
The coefficients w[1..d]= [[-0.29256139  0.18126728  1.74597664]]
[1 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0
 0 0 0 1 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 0 0
 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 0 0
 0 1 0 0 1 0 0 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0
 0 0 1 0 0 1 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0
 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 0 0 1 0
 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0
 0 1 0 1 0 0 0 1 0 1 0 0 0 1 1 1 0 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0]


c:\users\chrix\appdata\local\programs\python\python38-32\lib\site-packages\sklearn\utils\validation.py:72: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


result with l2 regularization

In [11]:
print(confusion_matrix(y_test, y_test_pred))
print(regr.score(x_test,y_test))
   print(classification_report(y_test, y_test_pred))

[[232  18]
 [ 97  86]]
0.7344110854503464
              precision    recall  f1-score   support

           0       0.71      0.93      0.80       250
           1       0.83      0.47      0.60       183

    accuracy                           0.73       433
   macro avg       0.77      0.70      0.70       433
weighted avg       0.76      0.73      0.72       433

